# TOPS Stack Notebook

## Introduction
In this notebook, we will run all steps of processing Tops Stack. 
The detailed algorithm for stack processing of TOPS data can be find here:

+ Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

### Define input parameters with default values

In [ ]:
# Parmeters for TOPS Stack processor

min_lat = 35.57793709766442
max_lat = 36.21619047354823
min_lon = -84.70058767311058
max_lon = -83.94773267597341

bbox = ""

master_date = ""

start_date = "2020-10-01"
end_date = "2020-10-31"

track_number = 7

# PCM-System Parameters
# These use reserved-prefix parameter names (_*) and are also parsed during `notebook-pge-wrapper specs` to generate the hysds-io and job-spec
_time_limit = 86400
_soft_time_limit = 86400
_disk_usage = "100GB"
_submission_type = "individual"
_required_queue = "factotum-job_worker-small"
_label = "topsStack Processor for HAMSAR"


In [ ]:
import os
import json

# if not already exists, create file _context.json with current parameters,
# needed by several tools below
d = {
"min_lat": min_lat,
"max_lat": max_lat,
"min_lon": min_lon,
"max_lon": max_lon,
"bbox": [],
"master_date": master_date,
"start_date": start_date,
"end_date": end_date,
"track_number": track_number
}

path = "./_context.json"
if os.path.exists(path):
    pass
else:
    print("create file %s with current parameters" % path)
    with open(path, "w") as fout:
        json.dump(d, fout, indent=4)

# if not already exists, create empty file _stdout.txt,
# required by ${toolDir}/07-create-datasets.sh
path = "./_stdout.txt"
if os.path.exists(path):
    pass
else:
    print("create empty file %s" % path)
    with open(path, "w") as fout:
        pass

### Processing Timing

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

# initialize state file
STATE_FILE_NAME=_state.json
echo '{}' > ./${STATE_FILE_NAME}

prefix="_x"
processingStart=$(date +%FT%T)
python ${toolDir}/state_rw.py write ${prefix}_processing_start $processingStart
echo "Processing start" $processingStart

### Prepare scenes

This step will downloads SLCs.

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/prepare_scenes.sh

### Get SLCs

This step gets SLCs

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/02-get-slcs.sh


### Fetch DEM

This step downloads DEMs.

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/03-get-dems.sh

### Create run scripts and fetch orbits

The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

`stackSentinel.py` generates all configuration and run files required to be executed on a stack of Sentinel-1 TOPS data. When stackSentinel.py is executed for a given workflow (-W option) a **configs** and **run_files** folder is generated. No processing is performed at this stage. Within the run_files folder different run\_#\_description files are contained which are to be executed as shell scripts in the run number order. Each of these run scripts call specific configure files contained in the “configs” folder which call ISCE in a modular fashion. The configure and run files will change depending on the selected workflow. To make run_# files executable, change the file permission accordingly (e.g., `chmod +x run_01_unpack_slc`).

```bash
stackSentinel.py -H     #To see workflow examples,
stackSentinel.py -h     #To get an overview of all the configurable parameters
```

Required parameters of stackSentinel.py include:

```cfg
-s SLC_DIRNAME          #A folder with downloaded Sentinel-1 SLC’s. 
-o ORBIT_DIRNAME        #A folder containing the Sentinel-1 orbits. Missing orbit files will be downloaded automatically
-a AUX_DIRNAME          #A folder containing the Sentinel-1 Auxiliary files
-d DEM_FILENAME         #A DEM (Digital Elevation Model) referenced to wgs84
```

In the following, different workflow examples are provided. Note that stackSentinel.py only generates the run and configure files. To perform the actual processing, the user will need to execute each run file in their numbered order.

In all workflows, coregistration (-C option) can be done using only geometry (set option = geometry) or with geometry plus refined azimuth offsets through NESD (set option = NESD) approach, the latter being the default. For the NESD coregistrstion the user can control the ESD coherence threshold (-e option) and the number of overlap interferograms (-O) to be used in NESD estimation.


In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/04-make-run-scripts.sh
sh ${toolDir}/05-make-run-scripts.sh


### Unpack TOPO reference

Unpacks the reference SLC files using ISCE readers, also unpackaging the antenna elevation pattern correction file if necessary. This run file also produces the reference geometry files that are consumed downstream.


In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-01.sh

### Unpack secondary SLC

Unpack Secondary SLCs
In a manner similar to the SLCs in the step above, this step unpacks the secondary SLCs from each of the input SLC zip files.

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-0.sh

### SLC noise calibration

This step runs radiometric and thermal noise calibration using vh-pol data against the SLCs.

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-5.sh

### Discard subswaths not to be used

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-02-9.sh

### Average baseline

Computes average baseline for the stack. These baselines are not used for processing anywhere. They are only an approximation and can be used for plotting purposes.

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-03.sh

### fullBurst geo2rdr

This step estimates geometrical offsets between secondary burst overlaps and the stack reference burst overlaps. The secondaries are then resampled to the stack reference burst overlaps.

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-04.sh

### fullBurst resample

Using orbit and DEM data, this run file computes geometrical offsets among all secondary SLCs and the stack reference. These offsets, with the misregistration time series are used for precise coregistration of each burst SLC.


In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-05.sh

### Extract stack valid region

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-06.sh

### Merge reference secondary SLC

This step merges all bursts for the reference and coregistered SLCs. Geometry files are also merged. Using orbit and DEM, geometrical offsets among all secondary SLCs and the stack reference is computed. The geometrical offsets, together with the misregistration time-series (from previous step) are used for precise coregistration of each burst SLC. 

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

sh ${toolDir}/06-run-07.sh

### Save result

The final step ‘packages’ up the results into a single sub-directory named as coregistered_slcs-<start_date>-<end_date> and generates json files describing the dataset (e.g. the polygon  coordinates of the bounding box) and associated metadata (e.g. included SLCs).

More information about metadata files can be found at : https://hysds-core.atlassian.net/wiki/spaces/HYS/pages/389349377/Product+Metadata+Files

In [ ]:
%%bash

toolDir=~/topsstack_hamsar/tool

source ${toolDir}/00-init.sh
source ${toolDir}/01-setup.sh

prefix="_x"
export PROCESSING_START=$(python ${toolDir}/state_rw.py read ${prefix}_processing_start)

sh ${toolDir}/07-create-datasets.sh